In [16]:
# imports dependacies 
import requests
import pandas as pd
from config import api_key
import pymongo 

In [29]:
conn = 'mongodb://localhost:27017'
# Pass connection to the pymongo instance.
client = pymongo.MongoClient(conn)
# Connects to austin_db
db = client.austin_db
# drops temp collection 
db.temp.drop()

In [3]:
# opens austin_weather in a dataframe 
csv_pd = pd.read_csv("austin_weather.csv")
csv_pd

,Date,TempHighF,TempAvgF,TempLowF,DewPointHighF,DewPointAvgF,DewPointLowF,HumidityHighPercent,HumidityAvgPercent,HumidityLowPercent,...,SeaLevelPressureAvgInches,SeaLevelPressureLowInches,VisibilityHighMiles,VisibilityAvgMiles,VisibilityLowMiles,WindHighMPH,WindAvgMPH,WindGustMPH,PrecipitationSumInches,Events
0,2013-12-21,74,60,45,67,49,43,93,75,57,...,29.68,29.59,10,7,2,20,4,31,0.46,"Rain , Thunderstorm"
1,2013-12-22,56,48,39,43,36,28,93,68,43,...,30.13,29.87,10,10,5,16,6,25,0,
2,2013-12-23,58,45,32,31,27,23,76,52,27,...,30.49,30.41,10,10,10,8,3,12,0,
3,2013-12-24,61,46,31,36,28,21,89,56,22,...,30.45,30.3,10,10,7,12,4,20,0,
4,2013-12-25,58,50,41,44,40,36,86,71,56,...,30.33,30.27,10,10,7,10,2,16,T,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1314,2017-07-27,103,89,75,71,67,61,82,54,25,...,29.97,29.88,10,10,10,12,5,21,0,
1315,2017-07-28,105,91,76,71,64,55,87,54,20,...,29.9,29.81,10,10,10,14,5,20,0,
1316,2017-07-29,107,92,77,72,64,55,82,51,19,...,29.86,29.79,10,10,10,12,4,17,0,
1317,2017-07-30,106,93,79,70,68,63,69,48,27,...,29.91,29.87,10,10,10,13,4,20,0,


In [15]:
# finds the mean of each temprature 
means = csv_pd.mean()
csv_dict = {
    "Temp": means[1],
    "TempHigh": means[0],
    "TempLow": means[2]
}
# stores means in a dictionary 
csv_dict

{'Temp': 70.6429112964367,
 'TempHigh': 80.86277482941622,
 'TempLow': 59.902956785443514}

In [21]:
# stores url for api weather app, unit, and city
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "imperial"
city = "Austin"
# builds query URL
query_url = f"{url}appid={api_key}&units={units}&q={city}"
# finds response
response = requests.get(query_url).json()

response

{'coord': {'lon': -97.74, 'lat': 30.27},
 'weather': [{'id': 803,
   'main': 'Clouds',
   'description': 'broken clouds',
   'icon': '04d'}],
 'base': 'stations',
 'main': {'temp': 78.44,
  'feels_like': 83.3,
  'temp_min': 77,
  'temp_max': 81,
  'pressure': 1015,
  'humidity': 78},
 'visibility': 10000,
 'wind': {'speed': 5.82, 'deg': 0},
 'clouds': {'all': 75},
 'dt': 1599070874,
 'sys': {'type': 1,
  'id': 3344,
  'country': 'US',
  'sunrise': 1599048528,
  'sunset': 1599094353},
 'timezone': -18000,
 'id': 4671654,
 'name': 'Austin',
 'cod': 200}

In [22]:
# stores temp info in to seprat dictionary 
api_dict = {
    "Temp": response["main"]["temp"],
    "TempHigh": response["main"]["temp_max"],
    "TempLow": response["main"]["temp_min"]
}

api_dict 

{'Temp': 78.44, 'TempHigh': 81, 'TempLow': 77}

In [32]:
# uploads dictinary to mongo db
db.temp.insert(csv_dict)

C:\Users\super\anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  """Entry point for launching an IPython kernel.


ObjectId('5f4fe39e760b20a0d3ed45ac')

In [33]:
# uploads dictinary to mongo db
db.temp.insert(api_dict)

C:\Users\super\anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  """Entry point for launching an IPython kernel.


ObjectId('5f4fe3b8760b20a0d3ed45ad')